# Self-Driving Car Engineer Nanodegree


## Project: **Finding Lane Lines on the Road** 
***
---

## Import Packages

In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

## Helper Functions

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [ ]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=8):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """

    # work with numpy
    arr = np.squeeze(np.array(lines))    
    # calculate the slope (y2-y1)/(x2-x1)
    slope = (arr[:,3]-arr[:,1])/(arr[:,2]-arr[:,0])
    # calculate b (y=mx+b)
    b = arr[:,1] - slope * arr[:,0]   
    
    # filter for relevant slopes
    absslope = np.abs(slope)
    intersect = np.intersect1d(np.where(absslope > .50), np.where(absslope < .85))
    filtered_slope = slope[intersect]
    filtered_b = b[intersect]
    
    # left lines
    left_indices = np.where(filtered_slope < 0)
    left_slope = np.average(filtered_slope[left_indices])
    left_b = np.average(filtered_b[left_indices])
    
    # right lines
    right_indices = np.where(filtered_slope > 0)
    right_slope = np.average(filtered_slope[right_indices])
    right_b = np.average(filtered_b[right_indices])

    if 'previous_values' in globals():
        global previous_values

        alpha = .7
        reci = 1.0 - alpha
        
        if previous_values[0] != 0:            
            left_slope = previous_values[0] * alpha + left_slope * reci
            left_b = previous_values[1] * alpha + left_b * reci

            right_slope = previous_values[2] * alpha + right_slope * reci
            right_b = previous_values[3] * alpha + right_b * reci
        
        previous_values[0] = left_slope
        previous_values[1] = left_b

        previous_values[2] = right_slope
        previous_values[3] = right_b
        
    pointl = lambda x: (x, int(x * left_slope + left_b))
    pointr = lambda x: (x, int(x * right_slope + right_b))

    size = img.shape[1]
    
    try:
        cv2.line(img, pointr(int(size/2)+20), pointr(size-10), color, thickness)
        cv2.line(img, pointl(10), pointl(int(size/2)-20), color, thickness)
    except (ValueError):
        print("there was an error in drawing the line")
        pass

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

def reset_previous():
    if 'previous_values' in globals():
        del globals()["previous_values"]
    global previous_values
    previous_values = np.zeros([4])

## Test Images

Build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [ ]:
import os
os.listdir("test_images/")

## Build a Lane Finding Pipeline



Build the pipeline and run your solution on all test_images. Make copies into the `test_images_output` directory, and you can use the images in your writeup report.

Try tuning the various parameters, especially the low and high Canny thresholds as well as the Hough lines parameters.

In [ ]:
for imf in os.listdir("test_images/"):
    print(imf)
   
    reset_previous()
    
    image = mpimg.imread('test_images/' + imf)

    # convert to grayscale
    gray = grayscale(image)

    # apply gaussian blur
    blurred = gaussian_blur(gray, 5)

    # canny transform
    edges = canny(blurred, 50, 150)

    # region of interest
    imshape = image.shape
    h = .62
    v = [.48, .52]
    vertices = np.array([[(0,imshape[0]),(imshape[1]*v[0], imshape[0]*h), (imshape[1]*v[1], imshape[0]*h), (imshape[1],imshape[0])]], dtype=np.int32)
    clipped = region_of_interest(edges, vertices)

    # hough transform
    # rho, theta, threshold, min_line_len, max_line_gap
    lines = hough_lines(clipped, 2, np.pi/180, 15, 20, 40)

    # create the final combined image
    final = weighted_img(lines, image) 
    
    mpimg.imsave(os.path.join("test_images_output/", imf), final)
    
    plt.figure()
    plt.imshow(final)

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

**Note: if you get an `import error` when you run the next cell, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt. Also, check out [this forum post](https://carnd-forums.udacity.com/questions/22677062/answers/22677109) for more troubleshooting tips.**

**If you get an error that looks like this:**
```
NeedDownloadError: Need ffmpeg exe. 
You can download it by calling: 
imageio.plugins.ffmpeg.download()
```
**Follow the instructions in the error message and check out [this forum post](https://carnd-forums.udacity.com/display/CAR/questions/26218840/import-videofileclip-error) for more troubleshooting tips across operating systems.**

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
def process_image(image):
    # convert to grayscale
    gray = grayscale(image)

    # apply gaussian blur
    blurred = gaussian_blur(gray, 5)

    # canny transform
    edges = canny(blurred, 50, 150)

    # region of interest
    imshape = image.shape
    h = .58
    v = [.48, .52]
    vertices = np.array([[(0,imshape[0]),(imshape[1]*v[0], imshape[0]*h), (imshape[1]*v[1], imshape[0]*h), (imshape[1],imshape[0])]], dtype=np.int32)
    clipped = region_of_interest(edges, vertices)

    # hough transform
    # rho, theta, threshold, min_line_len, max_line_gap
    lines = hough_lines(clipped, 2, np.pi/180, 15, 20, 40)

    # create the final combined image
    result = weighted_img(lines, image) 

    return result

Let's try the one with the solid white lane on the right first ...

In [ ]:
white_output = 'test_videos_output/solidWhiteRight.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
reset_previous()
    
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

## Improve the draw_lines() function

**At this point, if you were successful with making the pipeline and tuning parameters, you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform. As mentioned previously, try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines. You can see an example of the result you're going for in the video "P1_example.mp4".**

**Go back and modify your draw_lines function accordingly and try re-running your pipeline. The new output should draw a single, solid line over the left lane line and a single, solid line over the right lane line. The lines should start from the bottom of the image and extend out to the top of the region of interest.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [ ]:
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4').subclip(0,5)

reset_previous()

clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Writeup and Submission

If you're satisfied with your video outputs, it's time to make the report writeup in a pdf or markdown file. Once you have this Ipython notebook ready along with the writeup, it's time to submit for review! Here is a [link](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) to the writeup template file.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
challenge_output = 'test_videos_output/challenge.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip3 = VideoFileClip('test_videos/challenge.mp4').subclip(0,5)

reset_previous()

clip3 = VideoFileClip('test_videos/challenge.mp4')
challenge_clip = clip3.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))